# Small Excesses in ATLAS EWK analyses and pMSSM models
Scans using Small Excess criteria: good sensitivity with CLs_exp <= 0.05, not exluded for CLs_obs > 0.05.

First activate the environment.

In [1]:
conda init

no change     /opt/anaconda3/condabin/conda
no change     /opt/anaconda3/bin/conda
no change     /opt/anaconda3/bin/conda-env
no change     /opt/anaconda3/bin/activate
no change     /opt/anaconda3/bin/deactivate
no change     /opt/anaconda3/etc/profile.d/conda.sh
no change     /opt/anaconda3/etc/fish/conf.d/conda.fish
no change     /opt/anaconda3/shell/condabin/Conda.psm1
no change     /opt/anaconda3/shell/condabin/conda-hook.ps1
no change     /opt/anaconda3/lib/python3.12/site-packages/xontrib/conda.xsh
no change     /opt/anaconda3/etc/profile.d/conda.csh
no change     /Users/mamuzic/.bash_profile
No action taken.

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda activate seenv


CondaError: Run 'conda init' before 'conda activate'


Note: you may need to restart the kernel to use updated packages.


Next, prepare paths for the scan.

In [3]:
import os, sys
import se_helpers
my_dir = "/Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/SmallExcesses"
bino_csv = "../EWKpMSSM_HepDATA/Bino-DM.csv"
os.chdir(my_dir)

Now upload the CSV file.

In [4]:
df = se_helpers.load_scan(my_dir,bino_csv)

✅ Data loaded successfully!


Make some statistics:

In [5]:
n_models, n_info = df.shape
print("Total number of models:", n_models)
print("Number of info columns:", n_info)

columns = df.columns.tolist()
print("Columns:", columns)

Total number of models: 8897
Number of info columns: 37
Columns: ['Model_number', 'FullHad__ExpCLs', 'FullHad__ObsCLs', 'FullHad__level', '1Lbb__ExpCLs', '1Lbb__ObsCLs', '1Lbb__level', '2L0J__ExpCLs', '2L0J__ObsCLs', '2L0J__level', '2L2J__ExpCLs', '2L2J__ObsCLs', '2L2J__level', 'Compressed__ExpCLs', 'Compressed__ObsCLs', 'Compressed__level', '3LOffshell__ExpCLs', '3LOffshell__ObsCLs', '3LOffshell__level', '3LOnshell__ExpCLs', '3LOnshell__ObsCLs', '3LOnshell__level', '4L__ExpCLs', '4L__ObsCLs', '4L__level', 'DisappearingTrack__ObsCLs', 'DisappearingTrack__level', 'h_to_inv__ObsCLs', 'h_to_inv__level', 'mA__ObsCLs', 'mA__level', 'Final__CLs', 'Final__analysis', 'Final__level', 'Constraints__DM', 'Constraints__EW', 'Constraints__Flavour']


Extract analyses, consider only the ones with both ExpCLs and ObsCLs, consider the rest as additional constraints:

In [6]:
analyses = []
for col in columns:
    if col.endswith('__ExpCLs'):
        name = col.split('__')[0]
        if f"{name}__ObsCLs" in columns:
            analyses.append(name)

print("Analyses with ExpCLs or ObsCLs columns:")
print(analyses)

Analyses with ExpCLs or ObsCLs columns:
['FullHad', '1Lbb', '2L0J', '2L2J', 'Compressed', '3LOffshell', '3LOnshell', '4L']


Apply Small Excess criteria on the models, make a skim:

In [7]:
results = {}

for ana in analyses:
    exp_col = f"{ana}__ExpCLs"
    obs_col = f"{ana}__ObsCLs"

    if exp_col in df.columns and obs_col in df.columns:
        # Find models satisfying ExpCLs >= 0.05 and ObsCLs < 0.05
        mask = (df[exp_col] <= 0.05) & (df[obs_col] > 0.05)
        #subset = df.loc[mask, ["Model_number", exp_col, obs_col]]
        subset = df.loc[mask]
        
        results[ana] = subset
        print(f"\n🔹 {ana}: found {len(subset)} models with ExpCLs <= 0.05 and ObsCLs > 0.05")
    else:
        print(f"\n⚠️ Skipping {ana} (missing columns)")


🔹 FullHad: found 0 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 1Lbb: found 77 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 2L0J: found 2 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 2L2J: found 38 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 Compressed: found 24 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 3LOffshell: found 48 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 3LOnshell: found 44 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 4L: found 0 models with ExpCLs <= 0.05 and ObsCLs > 0.05


In [8]:
print(results["2L0J"].head())

      Model_number  FullHad__ExpCLs  FullHad__ObsCLs FullHad__level  \
26              26            0.627            0.551          Truth   
3580          3580            0.789            0.735          Truth   

      1Lbb__ExpCLs  1Lbb__ObsCLs 1Lbb__level  2L0J__ExpCLs  2L0J__ObsCLs  \
26           0.376         0.394       Truth        0.0260        0.0552   
3580         1.000         1.000       Truth        0.0351        0.0507   

     2L0J__level  ...  h_to_inv__ObsCLs  h_to_inv__level mA__ObsCLs  \
26         Truth  ...               0.0           Limits        1.0   
3580       Truth  ...               1.0           Limits        1.0   

      mA__level    Final__CLs Final__analysis  Final__level  Constraints__DM  \
26       Limits  8.050000e-13       3LOnshell         Truth              0.0   
3580     Limits  0.000000e+00       3LOnshell         Truth              1.0   

     Constraints__EW  Constraints__Flavour  
26               1.0                   1.0  
3580        

In [9]:
# Combine all excess models into one DataFrame
import pandas
excess_all = pandas.concat(results.values(), ignore_index=True)
excess_all = excess_all.drop_duplicates(subset=["Model_number"])
print(excess_all.head())

   Model_number  FullHad__ExpCLs  FullHad__ObsCLs FullHad__level  \
0            60            0.803            0.693          Truth   
1            82            0.216            0.095          Truth   
2           360            0.492            0.306          Truth   
3           465            1.000            1.000           Reco   
4           506            0.593            0.445          Truth   

   1Lbb__ExpCLs  1Lbb__ObsCLs 1Lbb__level  2L0J__ExpCLs  2L0J__ObsCLs  \
0      0.027845      0.135541        Reco       0.22700      0.197000   
1      0.036422      0.222229        Reco       0.22700      0.174000   
2      0.049202      0.161541        Reco       0.31000      0.310000   
3      0.033587      0.180966        Reco       0.25809      0.154642   
4      0.038890      0.264181        Reco       0.28200      0.186000   

  2L0J__level  ...  h_to_inv__ObsCLs  h_to_inv__level mA__ObsCLs  mA__level  \
0       Truth  ...               1.0           Limits        1.0     Limi

In [10]:
n_models, n_info = excess_all.shape
print("Number of SE models", n_models)

Number of SE models 226


In [11]:
base, ext = os.path.splitext(bino_csv)
se_bino_csv = my_dir+"/"+base + "_SE" + ext
excess_all.to_csv(se_bino_csv, index=False)
if os.path.exists(se_bino_csv):
    print("✅ Created Small Excesses skim csv file", se_bino_csv)
else:
    print("❌ Error producing the csv file.", se_bino_csv)

✅ Created Small Excesses skim csv file /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/SmallExcesses/../EWKpMSSM_HepDATA/Bino-DM_SE.csv


Now extract all SE modelid from CSV file:

In [12]:
se_helpers.extract_se(my_dir, bino_csv)
#ewk_csv = "../EWKpMSSM_HepDATA/EWKino.csv"
#se_helpers.extract_se(my_dir, ewk_csv)

✅ Data loaded successfully!
Total number of models: 8897
Number of info columns: 37
Analyses with ExpCLs or ObsCLs columns:
['FullHad', '1Lbb', '2L0J', '2L2J', 'Compressed', '3LOffshell', '3LOnshell', '4L']

🔹 FullHad: found 0 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 1Lbb: found 77 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 2L0J: found 2 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 2L2J: found 38 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 Compressed: found 24 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 3LOffshell: found 48 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 3LOnshell: found 44 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 4L: found 0 models with ExpCLs <= 0.05 and ObsCLs > 0.05
Number of SE models 226
✅ Created Small Excesses skim csv file /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/SmallExcesses/../EWKpMSSM_HepDATA/Bino-DM_SE.csv


,Model_number,FullHad__ExpCLs,FullHad__ObsCLs,FullHad__level,1Lbb__ExpCLs,1Lbb__ObsCLs,1Lbb__level,2L0J__ExpCLs,2L0J__ObsCLs,2L0J__level,...,h_to_inv__ObsCLs,h_to_inv__level,mA__ObsCLs,mA__level,Final__CLs,Final__analysis,Final__level,Constraints__DM,Constraints__EW,Constraints__Flavour
0,60,0.803,0.693,Truth,0.027845,0.135541,Reco,0.22700,0.197000,Truth,...,1.0,Limits,1.0,Limits,1.355407e-01,1Lbb,Reco,0.0,1.0,1.0
1,82,0.216,0.095,Truth,0.036422,0.222229,Reco,0.22700,0.174000,Truth,...,1.0,Limits,1.0,Limits,2.222290e-01,1Lbb,Reco,0.0,1.0,1.0
2,360,0.492,0.306,Truth,0.049202,0.161541,Reco,0.31000,0.310000,Truth,...,1.0,Limits,1.0,Limits,1.615406e-01,1Lbb,Reco,1.0,1.0,1.0
3,465,1.000,1.000,Reco,0.033587,0.180966,Reco,0.25809,0.154642,Reco,...,1.0,Limits,0.0,Limits,1.866409e-02,2L2J,Reco,0.0,1.0,0.0
4,506,0.593,0.445,Truth,0.038890,0.264181,Reco,0.28200,0.186000,Truth,...,1.0,Limits,1.0,Limits,2.344027e-02,3LOnshell,Reco,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,6016,1.000,1.000,Truth,0.786000,0.824000,Truth,0.72200,0.682000,Truth,...,1.0,Limits,0.0,Limits,0.000000e+00,mA,Limits,0.0,1.0,0.0
228,6500,0.789,0.668,Truth,0.185763,0.551196,Reco,0.26500,0.096100,Truth,...,1.0,Limits,1.0,Limits,6.083909e-02,3LOnshell,Reco,1.0,1.0,1.0
230,6955,0.879,0.789,Truth,0.224000,0.563000,Truth,0.48000,0.377000,Truth,...,1.0,Limits,1.0,Limits,5.340000e-02,3LOnshell,Truth,0.0,1.0,1.0
231,7141,1.000,1.000,Truth,0.998000,0.998000,Truth,1.00000,1.000000,Truth,...,1.0,Limits,0.0,Limits,2.350000e-13,3LOffshell,Truth,0.0,0.0,0.0


Now look up the SLHA of the model, and copy to a folder:

In [13]:
from pathlib import Path

slha_dir = Path("/Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM")
model = "191.slha"
modelid = 191

# Make new folder name with _SE suffix
se_slha_dir = slha_dir.parent / (slha_dir.name + "_SE")
se_slha_dir.mkdir(parents=True, exist_ok=True)

for f in slha_dir.rglob(model):
    print("✅ Found:", f)

    if False:
        with open(f, "r") as f:
            print(f.read())

    # Destination file path (same filename inside the new folder)
    dst = se_slha_dir / f.name

    # Copy file contents (no shutil)
    dst.write_bytes(f.read_bytes())

    print(f"✅ Copied {f.name} → {dst}")

✅ Found: /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM/0/191.slha
✅ Copied 191.slha → /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM_SE/191.slha


In [14]:
se_helpers.copy_model_slha(slha_dir, modelid)

✅ Found: /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM/0/191.slha
✅ Copied 191.slha to /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM_SE/191.slha


Now loop on all models from excess_all

In [15]:
# Loop on all models
for modelid in excess_all.Model_number:
    print(modelid)
    se_helpers.copy_model_slha(slha_dir, modelid)

60
✅ Found: /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM/0/60.slha
✅ Copied 60.slha to /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM_SE/60.slha
82
✅ Found: /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM/0/82.slha
✅ Copied 82.slha to /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM_SE/82.slha
360
✅ Found: /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM/0/360.slha
✅ Copied 360.slha to /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM_SE/360.slha
465
✅ Found: /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM/0/465.slha
✅ Copied 465.slha to /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM_SE/465.slha
506
✅ Found: /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_HepDATA/Bino-DM/0/506.slha
✅ Copied 506.slha to /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/EWKpMSSM_H

In [16]:
dirname = "/Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/SmallExcesses"
bino_csv = "../EWKpMSSM_HepDATA/Bino-DM.csv"
bino_slha_dir = "../EWKpMSSM_HepDATA/Bino-DM"

# load_scan(dirname, bino_csv) # Test
excess_all = se_helpers.extract_se(dirname, bino_csv)
se_helpers.copy_all_model_slha(dirname+"/"+bino_slha_dir, excess_all)

✅ Data loaded successfully!
Total number of models: 8897
Number of info columns: 37
Analyses with ExpCLs or ObsCLs columns:
['FullHad', '1Lbb', '2L0J', '2L2J', 'Compressed', '3LOffshell', '3LOnshell', '4L']

🔹 FullHad: found 0 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 1Lbb: found 77 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 2L0J: found 2 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 2L2J: found 38 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 Compressed: found 24 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 3LOffshell: found 48 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 3LOnshell: found 44 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 4L: found 0 models with ExpCLs <= 0.05 and ObsCLs > 0.05
Number of SE models 226
✅ Created Small Excesses skim csv file /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/SmallExcesses/../EWKpMSSM_HepDATA/Bino-DM_SE.csv
60
✅ Found: /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/SmallExcesses/../EWKpMSSM_HepDATA/Bino-DM/0/60.slh

Now repeat for EWK scan:

In [17]:
dirname = "/Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/SmallExcesses"
ewk_csv = "../EWKpMSSM_HepDATA/EWKino.csv"
ewk_slha_dir = "../EWKpMSSM_HepDATA/EWKino"

# load_scan(dirname, bino_csv) # Test
excess_all = se_helpers.extract_se(dirname, ewk_csv)
se_helpers.copy_all_model_slha(dirname+"/"+ewk_slha_dir, excess_all)

✅ Data loaded successfully!
Total number of models: 12280
Number of info columns: 37
Analyses with ExpCLs or ObsCLs columns:
['FullHad', '1Lbb', '2L0J', '2L2J', 'Compressed', '3LOffshell', '3LOnshell', '4L']

🔹 FullHad: found 0 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 1Lbb: found 98 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 2L0J: found 1 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 2L2J: found 12 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 Compressed: found 21 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 3LOffshell: found 1 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 3LOnshell: found 19 models with ExpCLs <= 0.05 and ObsCLs > 0.05

🔹 4L: found 0 models with ExpCLs <= 0.05 and ObsCLs > 0.05
Number of SE models 149
✅ Created Small Excesses skim csv file /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/SmallExcesses/../EWKpMSSM_HepDATA/EWKino_SE.csv
70
✅ Found: /Users/mamuzic/MyWork/LJUBLJANA/WP2_pMSSMML_SMASH/SmallExcesses/../EWKpMSSM_HepDATA/EWKino/0/70.slha
